<a href="https://colab.research.google.com/github/Bhavadharani275/Mini_Project_3/blob/main/Crop_Production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load a data

In [ ]:
# upload the FAOSTAT_data.xlsx in google drive then mount it in google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Load your Excel file
df = pd.read_excel("/content/drive/MyDrive/FAOSTAT_data.xlsx")

In [ ]:
df.head(5)

In [ ]:
df.shape

# cleaning process

In [ ]:
duplicates_shows = df[df.duplicated()]
print(duplicates_shows)

In [ ]:
# drop column
crop_data = df.drop(['Domain Code','Area Code (M49)','Element Code','Item Code (CPC)','Year Code','Flag',
       'Flag Description','Note'],axis=1)

In [ ]:
crop_data.duplicated().sum()

In [ ]:
crop_data.columns

In [ ]:
crop_data.shape

In [ ]:
crop_data.head(5)

In [ ]:
crop_data.dtypes

In [ ]:
crop_data.info()

In [ ]:
# Print number of missing values
missing_value_counts=crop_data.isnull().sum()
missing_value_counts

In [ ]:
# To see null row in id_method column
show=crop_data[crop_data['Value'].isnull()]
show.head(10)

In [ ]:
# unique in elemnt
crop_data['Element'].unique()

In [ ]:
crop_data['Item'].nunique()

In [ ]:
crop_data['Element'].value_counts()

In [ ]:
crop_data['Value'].isnull().groupby(crop_data['Element']).sum()

In [ ]:
crop_data_new=crop_data[crop_data['Element']=='Production'].reset_index(drop=True)
crop_data_new.head(5)

In [ ]:
df_filtered = crop_data[crop_data['Element'].isin(['Area harvested', 'Yield', 'Production'])]
print(df_filtered.shape)

# Transforming

In [ ]:
crop_predict = df_filtered.pivot_table(
    index=['Area', 'Item', 'Year'],
    columns='Element',
    values='Value'
).reset_index()

print(crop_predict.shape)

In [ ]:
crop_predict.rename(columns={
    'Area harvested': 'Area_harvested(ha)',
    'Yield': 'Yield(kg/ha)',
    'Production': 'Production(tons)'
}, inplace=True)
crop_predict.head(7)

In [ ]:
crop_predict.to_excel('crop_predict.xlsx', index=False)

In [ ]:
duplicate_sh = crop_predict[crop_predict.duplicated()]
print(duplicate_sh)

In [ ]:
crop_predict.dtypes

In [ ]:
print(crop_predict.isna().sum())

In [ ]:
crop_predict['Item'].unique()

In [ ]:
# delete the non-crop products
livestock_items = [
    'Butter and ghee of sheep milk', 'Butter of cow milk',
    'Whole milk, condensed', 'Whole milk, evaporated',
    'Cheese from milk of goats, fresh or processed',
    'Cheese from milk of sheep, fresh or processed',
    'Cheese from whole cow milk', 'Cheese from skimmed cow milk',
    'Ghee from cow milk', 'Butter of buffalo milk',
    'Raw milk of cattle', 'Raw milk of goats', 'Raw milk of sheep',
    'Raw milk of buffalo', 'Skim milk of cows', 'Skim milk, condensed',
    'Skim milk, evaporated', 'Whole milk powder', 'Whey, dry',
    'Whey, condensed', 'Yoghurt', 'Milk Animals'
]

In [ ]:
item_counts = crop_predict[crop_predict['Item'].isin(livestock_items)]['Item'].value_counts()

print(item_counts)

In [ ]:
total_livestock_rows = item_counts.sum()

print(total_livestock_rows)

In [ ]:
# unique word to remove non-crop product
crop_predict_new = crop_predict[~crop_predict['Item'].str.contains(
    'wine|beer|oil|cocoa|coffee|molasses|sugar|tobacco|tea|truffles|rubber|coir|jute|tallow|peppermint|spearmint|palm|palm kernel|palm oil|cottonseed oil|Shorn wool|Margarine and shortening|Cream, fresh|Raw silk|Silk-worm|Beeswax|honey|chickens|pig|snails|buffalo|milk|butter|ghee|cheese|whey|yoghurt|goat|sheep|camels|cattle|meat|eggs|pigs|Edible offals of horses and other equines', case=False, na=False
)]

In [ ]:
crop_predict_new.shape

In [ ]:
crop_predict_new['Item'].unique()

In [ ]:
print(crop_predict.isna().sum())
print(crop_predict.shape)

In [ ]:
crop_predict_new.info()

In [ ]:
print(crop_predict_new.isna().sum())
print(crop_predict_new.shape)

In [ ]:
# Assing 0 to production and yield if area harvested has 0
crop_predict_new.loc[crop_predict_new['Area_harvested(ha)'] == 0, ['Production(tons)', 'Yield(kg/ha)']] = 0

In [ ]:
crop_predict_new[crop_predict_new['Production(tons)'].isnull()]

In [ ]:
# True → production will be imputed
# False → production is original

crop_predict_new['Prod_imputed'] = crop_predict_new['Production(tons)'].isna()

In [ ]:
# Compute global averages for each crop
crop_avg = crop_predict_new.groupby('Item')[['Production(tons)', 'Yield(kg/ha)']].mean()

# Fill missing Production using global crop average
crop_predict_new['Production(tons)'] = crop_predict_new.apply(
    lambda row: crop_avg.loc[row['Item'], 'Production(tons)']
                if pd.isna(row['Production(tons)']) else row['Production(tons)'],
    axis=1
)

# Fill Yield using same approach
crop_predict_new['Yield(kg/ha)'] = crop_predict_new.apply(
    lambda row: crop_avg.loc[row['Item'], 'Yield(kg/ha)']
                if pd.isna(row['Yield(kg/ha)']) else row['Yield(kg/ha)'],
    axis=1
)

In [ ]:
crop_predict_new[crop_predict_new['Production(tons)'].isnull()]

In [ ]:
crop_predict_new[crop_predict_new['Yield(kg/ha)'].isnull()]

In [ ]:
items_with_nan_yield = crop_predict_new[crop_predict_new['Yield(kg/ha)'].isna()]['Item'].unique()
print(items_with_nan_yield)

In [ ]:
# Identify items where all rows have NaN for Yield and Area
items_all_nan = crop_predict_new.groupby('Item').apply(
    lambda x: x['Yield(kg/ha)'].isna().all() and x['Area_harvested(ha)'].isna().all()
).loc[lambda x: x].index.tolist()

# Set a placeholder Yield (example: 1000 kg/ha)
placeholder_yield = 1000

# Impute Yield for these items
crop_predict_new.loc[crop_predict_new['Item'].isin(items_all_nan), 'Yield(kg/ha)'] = placeholder_yield

# Compute Area if Production exists
crop_predict_new.loc[crop_predict_new['Item'].isin(items_all_nan) & crop_predict_new['Production(tons)'].notna(), 'Area_harvested(ha)'] = \
    crop_predict_new['Production(tons)'] * 1000 / placeholder_yield

# Flag these rows as imputed
crop_predict_new['area_imputed'] = crop_predict_new.get('area_imputed', False)  # create column if not exists
crop_predict_new['yield_imputed'] = crop_predict_new.get('yield_imputed', False)  # create column if not exists
# crop_predict_new.loc[crop_predict_new['Item'].isin(items_all_nan), 'imputed'] = True

In [ ]:
crop_predict_new[crop_predict_new['Yield(kg/ha)'].isnull()]

In [ ]:
crop_predict_new[crop_predict_new['Area_harvested(ha)'].isnull()]

In [ ]:
items_with_nan_area = crop_predict_new[crop_predict_new['Area_harvested(ha)'].isna()]['Item'].unique()
print(items_with_nan_area)

In [ ]:
# Compute missing Area_harvested where Yield and Production exist
mask_area_missing = crop_predict_new['Area_harvested(ha)'].isna() & crop_predict_new['Yield(kg/ha)'].notna() & crop_predict_new['Production(tons)'].notna()

crop_predict_new.loc[mask_area_missing, 'Area_harvested(ha)'] = \
   crop_predict_new.loc[mask_area_missing, 'Production(tons)'] * 1000 / crop_predict_new.loc[mask_area_missing, 'Yield(kg/ha)']

# Flag these rows as imputed for Area
crop_predict_new['area_imputed'] =crop_predict_new.get('area_imputed', False)
crop_predict_new.loc[mask_area_missing, 'area_imputed'] = True




In [ ]:
crop_predict_new[crop_predict_new['Area_harvested(ha)'].isnull()]

In [ ]:
crop_predict_new.to_excel('crop_predict_new.xlsx', index=False)

In [ ]:
crop_predict_new[['Area_harvested(ha)', 'Production(tons)', 'Yield(kg/ha)']].describe()

In [ ]:
crop_predict_new["Area_harvested(ha)"].max()

In [ ]:
# Replace misencoded names in 'Area' column
crop_predict_new['Area'] = crop_predict_new['Area'].replace({
    "C�te d'Ivoire": "Cote d'Ivoire",
    "T�rkiye": "Turkiye"
})

# Replace misencoded names in 'Item' column
crop_predict_new['Item'] = crop_predict_new['Item'].replace({
    "Mat� leaves": "Mate leaves"
})

# Verify replacements
print(crop_predict_new.loc[
    (crop_predict_new['Area'].isin(["Cote d'Ivoire", "Turkiye"])) |
    (crop_predict_new['Item'].isin(["Mate leaves"])),
    ['Area', 'Item']
].drop_duplicates())

In [ ]:
Production_df = crop_predict_new[
    (crop_predict_new['Area_harvested(ha)'] > 0) &
    (crop_predict_new['Yield(kg/ha)'] > 0) &
    (~crop_predict_new['Area_harvested(ha)'].isna()) &
    (~crop_predict_new['Yield(kg/ha)'].isna())
].copy()


In [ ]:
crop_predict_new.shape

In [ ]:
Production_df.shape

In [ ]:
Production_df["Item"].unique()

In [ ]:
print(Production_df["Area_harvested(ha)"].min())
print(Production_df["Area_harvested(ha)"].max())
print(crop_predict_new["Area_harvested(ha)"].min())
print(crop_predict_new["Area_harvested(ha)"].max())

In [ ]:
Production_df.head()

In [ ]:
# final dataset
Production_df.to_excel('crop_production.xlsx', index=False)

In [ ]:
Production_df["Item"].unique()

# Distributions

In [ ]:
import plotly.express as px

num_cols = ['Area_harvested(ha)', 'Yield(kg/ha)', 'Production(tons)']

for col in num_cols:
    fig = px.histogram(Production_df, x=col, nbins=50, title=f'Distribution of {col}')
    fig.show()

In [ ]:
fig = px.histogram(Production_df, x='Production(tons)', nbins=50, log_y=True, title='Log-scaled Production Distribution')
fig.show()

# Correlations

In [ ]:
import plotly.figure_factory as ff
import numpy as np

# Compute correlation matrix
corr = Production_df[num_cols].corr().round(2)

# Interactive heatmap
fig = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.index),
    colorscale='Viridis',
    showscale=True
)
fig.update_layout(title='Correlation Heatmap')
fig.show()


# Relationships / Scatter Plots

In [ ]:
# Area vs Production
fig = px.scatter(Production_df, x='Area_harvested(ha)', y='Production(tons)',
                 color='Item', hover_data=['Year', 'Area'], title='Area vs Production')

fig.update_layout(height=600, width=2000)
fig.show()

# Yield vs Production
fig = px.scatter(Production_df, x='Yield(kg/ha)', y='Production(tons)',
                 color='Item', hover_data=['Year', 'Area'], title='Yield vs Production')

fig.update_layout(height=600, width=2000)
fig.show()

# Analyze Crop Distribution

## Crop Types

### most cultivated crops

In [ ]:
import plotly.express as px

# Count the occurrences of each crop type
crop_counts = Production_df['Item'].value_counts().reset_index()
crop_counts.columns = ['Item', 'Count']

# Top 15 most cultivated crops
top_crops = crop_counts.head(20)

# Plotly bar chart
fig = px.bar(
    top_crops,
    x='Item',
    y='Count',
    title='Top 15 Most Cultivated Crop Types Across Regions',
    text='Count',
    color='Count',
    color_continuous_scale='Viridis'
)

fig.update_layout(
    xaxis_title="Crop Type (Item)",
    yaxis_title="Number of Records",
    xaxis_tickangle=45,
    title_x=0.5
)

fig.show()


### least cultivated crops

In [ ]:
# Bottom 15 least cultivated crops
least_crops = crop_counts.tail(15)

fig = px.bar(
    least_crops,
    x='Item',
    y='Count',
    title='Least Cultivated Crop Types Across Regions',
    text='Count',
    color='Count',
    color_continuous_scale='Reds'
)

fig.update_layout(
    xaxis_title="Crop Type (Item)",
    yaxis_title="Number of Records",
    xaxis_tickangle=45,
    title_x=0.5
)

fig.show()


### Region-Wise Crop Distribution (Heatmap)

In [ ]:
import plotly.express as px
import pandas as pd

# Create a pivot table: Countries vs Crops
region_crop = (
    Production_df
    .groupby(['Area', 'Item'])
    .size()
    .reset_index(name='Count')
)

# Filter top 20 crops for clarity
top_items = region_crop['Item'].value_counts().head(20).index
region_crop_filtered = region_crop[region_crop['Item'].isin(top_items)]

# Plotly heatmap
fig = px.density_heatmap(
    region_crop_filtered,
    x='Item',
    y='Area',
    z='Count',
    color_continuous_scale='Viridis',
    title='🌍 Region-wise Distribution of Top 20 Crops',
)

fig.update_layout(
    xaxis_title="Crop Type (Item)",
    yaxis_title="Country (Area)",
    title_x=0.5,
    height=1000,
    width=1500)

fig.show()


## Geographical Distribution

### Agricultural Production by Country

In [ ]:
import plotly.express as px
import pandas as pd

# Aggregate production by country
geo_df = (
    Production_df.groupby('Area', as_index=False)['Production(tons)']
    .sum()
    .sort_values(by='Production(tons)', ascending=False)
)

# Plot Choropleth Map
fig = px.choropleth(
    geo_df,
    locations='Area',
    locationmode='country names',
    color='Production(tons)',
    hover_name='Area',
    color_continuous_scale='YlGn',
    title='🌾 Global Agricultural Production by Country'
)

fig.update_layout(
    title_x=0.5,
    geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth')
)
fig.show()


### Top Agricultural Regions

In [ ]:
fig = px.bar(
    geo_df.head(20),
    x='Area',
    y='Production(tons)',
    color='Production(tons)',
    title='🏆 Top 15 Agricultural Producers',
    color_continuous_scale='Greens'
)

fig.update_layout(
    xaxis_title='Country',
    yaxis_title='Total Production (tons)',
    title_x=0.5
)
fig.show()


### Crop Specialization by Region

In [ ]:
import plotly.express as px

# Aggregate by Area and Item
region_crop_prod = (
    Production_df.groupby(['Area', 'Item'], as_index=False)['Production(tons)'].sum()
)

# Keep top 20 countries for readability
top_countries = region_crop_prod.groupby('Area')['Production(tons)'].sum().nlargest(20).index
region_crop_prod = region_crop_prod[region_crop_prod['Area'].isin(top_countries)]

# Scatter plot
fig = px.scatter(
    region_crop_prod,
    x="Area",
    y="Production(tons)",
    color="Item",
    size="Production(tons)",
    hover_name="Item",
    title="🌾 Crop Specialization by Region (Top 20 Countries)",
)

fig.update_layout(
    title_x=0.5,
    xaxis_title="Country",
    yaxis_title="Total Production (tons)",
    xaxis_tickangle=45,
    plot_bgcolor="white",
    height=600, width=1500
)

fig.show()


# Temporal Analysis

### Yearly Trends: Area, Yield & Production

In [ ]:
import plotly.express as px
import pandas as pd

# Group by Year and aggregate numeric columns
yearly_trends = (
    Production_df.groupby('Year', as_index=False)[
        ['Area_harvested(ha)', 'Yield(kg/ha)', 'Production(tons)']
    ].mean()
)

# Melt the DataFrame for easy plotting
yearly_melted = yearly_trends.melt(id_vars='Year',
                                   value_vars=['Area_harvested(ha)', 'Yield(kg/ha)', 'Production(tons)'],
                                   var_name='Metric',
                                   value_name='Value')

# Line plot
fig = px.line(
    yearly_melted,
    x='Year',
    y='Value',
    color='Metric',
    markers=True,
    title="📈 Yearly Trends in Area Harvested, Yield, and Production"
)

fig.update_layout(
    title_x=0.5,
    xaxis_title="Year",
    yaxis_title="Average Value",
    plot_bgcolor="white",
    hovermode="x unified"
)

fig.show()



### Growth Analysis

#### Crop-Wise Growth Trend

In [ ]:
import pandas as pd
import plotly.express as px

# Aggregate by Item (crop) and Year
crop_growth = (
    Production_df.groupby(['Item', 'Year'], as_index=False)[['Yield(kg/ha)', 'Production(tons)']].mean()
)

# Example: show only top 10 crops (by total production)
top_crops = (
    crop_growth.groupby('Item')['Production(tons)']
    .sum()
    .nlargest(20)
    .index
)
crop_growth = crop_growth[crop_growth['Item'].isin(top_crops)]

# Line plot for yield trend
fig = px.line(
    crop_growth,
    x='Year',
    y='Yield(kg/ha)',
    color='Item',
    title="🌾 Crop-wise Yield Trends (Top 10 Crops)",
    markers=True
)
fig.update_layout(title_x=0.5, plot_bgcolor="white")
fig.show()

# Line plot for production trend
fig = px.line(
    crop_growth,
    x='Year',
    y='Production(tons)',
    color='Item',
    title="🌾 Crop-wise Production Trends (Top 10 Crops)",
    markers=True
)
fig.update_layout(title_x=0.5, plot_bgcolor="white")
fig.show()


#### Region-wise Production Trends

In [ ]:
# Aggregate by Area (country) and Year
region_growth = (
    Production_df.groupby(['Area', 'Year'], as_index=False)[['Yield(kg/ha)', 'Production(tons)']].mean()
)

# Top 10 countries by total production
top_regions = region_growth.groupby('Area')['Production(tons)'].sum().nlargest(10).index
region_growth = region_growth[region_growth['Area'].isin(top_regions)]

# Plot
fig = px.line(
    region_growth,
    x='Year',
    y='Production(tons)',
    color='Area',
    title="🌍 Region-wise Production Trends (Top 10 Countries)",
    markers=True
)
fig.update_layout(title_x=0.5, plot_bgcolor="white")
fig.show()



#  Infer relationships

## Correlation Between Area, Yield, and Production

In [ ]:
import plotly.figure_factory as ff

corr = Production_df[['Area_harvested(ha)', 'Yield(kg/ha)', 'Production(tons)']].corr().round(2)

fig = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.columns),
    colorscale='Viridis',
    showscale=True
)

fig.update_layout(title="📈 Correlation Between Area, Yield, and Production", title_x=0.5)
fig.show()


# Input-Output Relationships

In [ ]:
import pandas as pd

# Select relevant columns
cols = ['Area_harvested(ha)', 'Yield(kg/ha)', 'Production(tons)']
corr_matrix = Production_df[cols].corr()

print(corr_matrix)


In [ ]:
import plotly.express as px

# Production vs Area harvested
fig1 = px.scatter(Production_df, x='Area_harvested(ha)', y='Production(tons)',
                  title='Production vs Area Harvested', trendline='ols')
fig1.show()

# Production vs Yield
fig2 = px.scatter(Production_df, x='Yield(kg/ha)', y='Production(tons)',
                  title='Production vs Yield', trendline='ols')
fig2.show()


# Comparative Analysis

## Across Crops

In [ ]:
import pandas as pd

crop_yield_stats = (
    Production_df
    .groupby('Item')['Yield(kg/ha)']
    .agg(['mean', 'median', 'std', 'count'])
    .sort_values(by='mean', ascending=False)
)

print(crop_yield_stats.head(10))  # Top 10 high-yield crops
print(crop_yield_stats.tail(10))  # Bottom 10 low-yield crops


### Distribution of Yield per Crop

In [ ]:
#Boxplot
import plotly.express as px

# Optional: select top 20 crops for clarity
top_crops = Production_df['Item'].value_counts().head(20).index
df_top_crops = Production_df[crop_predict_new['Item'].isin(top_crops)]

fig = px.box(
    df_top_crops,
    x='Item',
    y='Yield(kg/ha)',
    color='Item',
    title='Crop-Wise Yield Distribution',
    points='all'  # 'outliers' to show only outliers
)

fig.update_layout(
    xaxis_title="Crop",
    yaxis_title="Yield (kg/ha)",
    xaxis_tickangle=-45,
    showlegend=False,
    title_x=0.5,
    height=1000,
    width=1500)
fig.show()


### Mean Yield per Crop

In [ ]:
mean_yield = Production_df.groupby('Item')['Yield(kg/ha)'].mean().sort_values(ascending=False)

fig = px.bar(
    mean_yield.head(20),  # Top 20 high-yield crops
    x=mean_yield.head(20).values,
    y=mean_yield.head(20).index,
    orientation='h',
    title='Top 20 High-Yield Crops',
    labels={'x':'Mean Yield (kg/ha)', 'y':'Crop'}
)
fig.update_layout(title_x=0.5)
fig.show()


## Across Regions

In [ ]:
import pandas as pd

# Sum production per region
region_production = (
    Production_df
    .groupby('Area')['Production(tons)']
    .sum()
    .sort_values(ascending=False)
)

print(region_production.head(10))  # Top 10 highly productive regions


### Top Regions

In [ ]:
import plotly.express as px

# Top 20 regions for clarity
top_regions = region_production.head(20)

fig = px.bar(
    x=top_regions.values,
    y=top_regions.index,
    orientation='h',
    title='Top 20 Highly Productive Regions',
    labels={'x':'Total Production (tons)', 'y':'Region'}
)

fig.update_layout(title_x=0.5)
fig.show()


### Production by Region vs Crop

In [ ]:
# Aggregate by region and crop
region_crop_prod = (
    Production_df.groupby(['Area', 'Item'])['Production(tons)'].sum().reset_index()
)

# Filter top 20 crops for clarity
top_crops = region_crop_prod['Item'].value_counts().head(20).index
region_crop_filtered = region_crop_prod[region_crop_prod['Item'].isin(top_crops)]

fig = px.density_heatmap(
    region_crop_filtered,
    x='Item',
    y='Area',
    z='Production(tons)',
    color_continuous_scale='Viridis',
    title='🌍 Region vs Crop Production Heatmap (Top 20 Crops)'
)

fig.update_layout(
    xaxis_title="Crop Type (Item)",
    yaxis_title="Region",
    title_x=0.5,
    height=1000,
    width=1500
)

fig.show()


## Productivity Analysis

### Compare Productivity Across Crops

In [ ]:
import plotly.express as px

top_crops = Production_df['Item'].value_counts().head(20).index
df_top_crops = Production_df[Production_df['Item'].isin(top_crops)]

fig = px.box(
    df_top_crops,
    x='Item',
    y='Yield(kg/ha)',
    color='Item',
    points='all',  # Show all data points
    title='🌱 Crop-Wise Yield Distribution'
)

fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title="Crop Type",
    yaxis_title="Yield (kg/ha)",
    showlegend=False,
    title_x=0.5,
    height=900,
    width=1000
)
fig.show()


### Compare Productivity Across Regions

In [ ]:
top_regions = Production_df['Area'].value_counts().head(20).index
df_top_regions = Production_df[Production_df['Area'].isin(top_regions)]

fig = px.box(
    df_top_regions,
    x='Area',
    y='Yield(kg/ha)',
    color='Area',
    points='all',
    title='🌍 Regional Yield Distribution (Top 20 Countries)'
)

fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title="Region",
    yaxis_title="Yield (kg/ha)",
    showlegend=False,
    title_x=0.5
)
fig.show()


# outliers detection

In [ ]:
def count_outliers(crop_predict_new, columns):
    outlier_counts = {}

    for col in columns:
        Q1 = crop_predict_new[col].quantile(0.25)
        Q3 = crop_predict_new[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outliers = crop_predict_new[(crop_predict_new[col] < lower) | (crop_predict_new[col] > upper)]
        outlier_counts[col] = len(outliers)
        print(f"{col}: {len(outliers)} outliers")

    return outlier_counts

# Example usage
num_cols = ['Area_harvested(ha)', 'Yield(kg/ha)', 'Production(tons)']
outlier_summary = count_outliers(crop_predict_new, num_cols)


In [ ]:
import plotly.express as px

# Boxplot for Area_harvested
fig = px.box(crop_predict_new, y='Area_harvested(ha)', points='all', title='Area_harvested Outliers')
fig.show()

# Boxplot for Production
fig = px.box(crop_predict_new, y='Production(tons)', points='all', title='Production Outliers')
fig.show()

# Boxplot for Yield
fig = px.box(crop_predict_new, y='Yield(kg/ha)', points='all', title='Yield Outliers')
fig.show()

In [ ]:
Production_df.head(6)

# Encoding the categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Copy the original dataset
encoded_df = Production_df.copy()

# Initialize LabelEncoders for categorical columns
le_area = LabelEncoder()
le_item = LabelEncoder()

# Encode categorical columns
encoded_df['Area'] = le_area.fit_transform(encoded_df['Area'])
encoded_df['Item'] = le_item.fit_transform(encoded_df['Item'])

# Encode Boolean columns (convert True/False → 1/0)
bool_cols = ['Prod_imputed', 'area_imputed', 'yield_imputed']
encoded_df[bool_cols] = encoded_df[bool_cols].astype(int)

# ✅ Verification
# print(encoded_df.dtypes)
# print(encoded_df.head())


# Regression Model

## Define Features and Target

In [ ]:
X = encoded_df[['Area', 'Item', 'Year',
                'Area_harvested(ha)', 'Yield(kg/ha)',
                'Prod_imputed', 'area_imputed', 'yield_imputed']]

y = encoded_df['Production(tons)']

## Split the Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

## Standardize Numeric Features

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_test_scaled

### Linear model

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

model = LinearRegression()
model.fit(X_train_scaled, y_train)



In [ ]:
model

In [ ]:
y_pred = model.predict(X_test_scaled)
print(y_pred)

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

# Example: y_test and y_pred are your actual and predicted values
# (Replace with your real arrays)
# y_pred = model.predict(X_test_scaled)
# y_pred = np.maximum(y_pred, 0)  # ensure no negatives if needed

# Create DataFrame for plotting
results_df = pd.DataFrame({
    "Actual Production": y_test,
    "Predicted Production": y_pred
})

# Scatter plot
fig = px.scatter(
    results_df,
    x="Actual Production",
    y="Predicted Production",
    trendline="ols",
    opacity=0.7,
    title="📊 Actual vs Predicted Production (Model Performance)",
    labels={
        "Actual Production": "Actual Production (tons)",
        "Predicted Production": "Predicted Production (tons)"
    },
    color_discrete_sequence=["#1f77b4"]
)

# Add a perfect-fit diagonal line
min_val = min(results_df["Actual Production"].min(), results_df["Predicted Production"].min())
max_val = max(results_df["Actual Production"].max(), results_df["Predicted Production"].max())

fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", dash="dash"),
    name="Perfect Fit"
)

fig.update_layout(
    title_x=0.5,
    width=800,
    height=600,
    template="plotly_white"
)

fig.show()


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score: {r2:.3f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")


# 5 Regression Models

## log-transform

In [ ]:
import numpy as np

# Apply log-transform
encoded_df['log_Area'] = np.log1p(encoded_df['Area_harvested(ha)'])
encoded_df['log_Yield'] = np.log1p(encoded_df['Yield(kg/ha)'])
encoded_df['log_Production'] = np.log1p(encoded_df['Production(tons)'])


In [ ]:
# final encoded dataset
encoded_df.to_excel('crop_encoded.xlsx', index=False)

In [ ]:
encoded_df.head()

# Split, Train, Predict, Evaluate and Save trained model


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle

# -----------------------------
# Load dataset
# -----------------------------
encoded_df = pd.read_excel("crop_encoded.xlsx")  # Must include log_Production, log_Area, log_Yield

X = encoded_df[['Area', 'Item', 'Year', 'log_Area', 'log_Yield']]
y = encoded_df['Production(tons)']  # Raw production in tons for evaluation

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Log-transform target for training
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# -----------------------------
# Feature scaling (for SVR & KNN)
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# Define models
# -----------------------------
models = {
    "LinearRegression": LinearRegression(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor()
}

# -----------------------------
# Train, predict, evaluate
# -----------------------------
results = []

for name, model in models.items():
    # Use scaled features for SVR and KNN
    if name in ["SVR", "KNN"]:
        model.fit(X_train_scaled, y_train_log)
        y_pred_log = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train_log)
        y_pred_log = model.predict(X_test)

    # Convert back to actual tons
    y_pred_actual = np.expm1(y_pred_log)

    # Evaluate in actual tons
    mae = mean_absolute_error(y_test, y_pred_actual)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_actual))
    r2 = r2_score(y_test, y_pred_actual)

    print(f"{name} → R²: {r2:.3f}, MAE: {mae:,.0f}, RMSE: {rmse:,.0f}")

    # Save trained model
    filename = f"{name}_model.pkl"
    with open(filename, "wb") as f:
        pickle.dump(model, f)

    results.append({
        "Model": name,
        "R2": r2,
        "MAE": mae,
        "RMSE": rmse
    })

# -----------------------------
# Summary
# -----------------------------
metrics_df = pd.DataFrame(results)
print("\nAll model metrics:")
print(metrics_df)


## Best and worst model

In [ ]:
from math import e
import pandas as pd

results = metrics_df

# Find best and worst models
best_model = results.loc[results['R2'].idxmax()]
worst_model = results.loc[results['R2'].idxmin()]

print("✅ Best Model:")
print(best_model, "\n")

print("❌ Worst Model:")
print(worst_model)


## Plot Evaluation Metrics

In [ ]:
import plotly.graph_objects as go
import pandas as pd

results = metrics_df

# Create grouped bar chart
fig = go.Figure(data=[
    go.Bar(name='MAE', x=results['Model'], y=results['MAE']),
    go.Bar(name='RMSE', x=results['Model'], y=results['RMSE']),
    go.Bar(name='R²', x=results['Model'], y=results['R2'])
])

fig.update_layout(
    barmode='group',
    title='Model Performance Comparison',
    xaxis_title='Models',
    yaxis_title='Score / Error',
    width=900,
    height=500
)

fig.show()


## Plot Actual vs Predicted values for all models

In [ ]:
import plotly.graph_objects as go

# Create Plotly figure
fig = go.Figure()

# Loop through models again for visualization
for name, model in models.items():
    # Predict (scaled for SVR & KNN)
    if name in ["SVR", "KNN"]:
        y_pred_log = model.predict(X_test_scaled)
    else:
        y_pred_log = model.predict(X_test)

    # Convert predictions back to actual scale
    y_pred_actual = np.expm1(y_pred_log)

    # Sort for cleaner visualization
    sorted_idx = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[sorted_idx]
    y_pred_sorted = y_pred_actual[sorted_idx]

    # Add each model’s line
    fig.add_trace(go.Scatter(
        x=y_test_sorted,
        y=y_pred_sorted,
        mode='lines+markers',
        visible='legendonly',  # 🔹 also hidden until clicked
        name=f"{name}",
        line=dict(width=2),
        marker=dict(size=5),
        hovertemplate="<b>Actual:</b> %{x:,.0f}<br><b>Predicted:</b> %{y:,.0f}"
    ))

# Add perfect prediction reference line (y = x)
fig.add_trace(go.Scatter(
    x=y_test,
    y=y_test,
    mode='lines',
    name="Perfect Prediction (y=x)",
    line=dict(color='black', dash='dot')
))

# Customize layout
fig.update_layout(
    title="📈 Actual vs Predicted Crop Production for All Models",
    xaxis_title="Actual Production (tons)",
    yaxis_title="Predicted Production (tons)",
    template="plotly_white",
    legend_title="Model",
    width=950,
    height=600
)

fig.show()


## Added area and item name in dataset

In [ ]:
import pandas as pd

# Load both files
original = pd.read_excel("crop_production.xlsx")
encoded = pd.read_excel("crop_encoded.xlsx")

# Add original text columns to encoded dataframe
encoded['Area_name'] = original['Area']
encoded['Item_name'] = original['Item']

# Save the merged file
encoded.to_excel("crop_encoded.xlsx", index=False)

print("✅ Added original Area and Item names to encoded file successfully!")


# Streamlit app

In [ ]:
# %%writefile app.py

In [ ]:
code = """
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import datetime
import plotly.graph_objects as go
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Load dataset and trained models
# -------------------------------
@st.cache_data
def load_data():
    return pd.read_excel("crop_encoded.xlsx")  # Must include Area_name, Item_name, log_Production etc.

@st.cache_resource
def load_models():
    model_paths = {
        "LinearRegression": "LinearRegression_model.pkl",
        "RandomForest": "RandomForest_model.pkl",
        "DecisionTree": "DecisionTree_model.pkl",
        "SVR": "SVR_model.pkl",
        "KNN": "KNN_model.pkl"
    }
    models = {}
    for name, path in model_paths.items():
        with open(path, "rb") as f:
            models[name] = pickle.load(f)
    return models

# -------------------------------
# Initialize data and models
# -------------------------------
data = load_data()
models_dict = load_models()

# -------------------------------
# Sidebar Inputs
# -------------------------------
st.sidebar.header("🧭 Input Parameters")

mapping_area = dict(zip(data['Area_name'], data['Area']))
mapping_item = dict(zip(data['Item_name'], data['Item']))

area_input = st.sidebar.selectbox("🌍 Select Country/Region", list(mapping_area.keys()))
item_input = st.sidebar.selectbox("🌾 Select Crop Item", list(mapping_item.keys()))

current_year = datetime.datetime.now().year
year_input = st.sidebar.number_input(
    "📅 Select Year",
    min_value=int(data['Year'].min()),
    max_value=current_year + 10,  # allows selecting up to 10 years into the future
    value=current_year
)

# Historical values
hist_values = data[
    (data['Area'] == mapping_area[area_input]) &
    (data['Item'] == mapping_item[item_input]) &
    (data['Year'] == year_input)
]

# Inputs for area and yield
area_value = st.sidebar.number_input(
    "Enter Area Harvested (ha)",
    value=float(hist_values['Area_harvested(ha)'].values[0]) if not hist_values.empty else 0
)

yield_value = st.sidebar.number_input(
    "Enter Yield (kg/ha)",
    value=float(hist_values['Yield(kg/ha)'].values[0]) if not hist_values.empty else 0
)

model_input = st.sidebar.selectbox("🤖 Select Model", list(models_dict.keys()))
predict_button = st.sidebar.button("🔮 Predict Production")

# -------------------------------
# Main UI
# -------------------------------
st.title("🌾 Crop Production Predictor Dashboard")
st.subheader("📊 Prediction Results & Visualization")

if predict_button:
    model = models_dict[model_input]

    # -------------------------------
    # Encode input
    # -------------------------------
    input_encoded = pd.DataFrame({
        'Area': [mapping_area[area_input]],
        'Item': [mapping_item[item_input]],
        'Year': [year_input],
        'log_Area': [np.log1p(area_value)],
        'log_Yield': [np.log1p(yield_value)]
    })

    # -------------------------------
    # Optional: Scale log features for SVR/KNN
    # -------------------------------
    if model_input in ['SVR', 'KNN']:
        scaler = StandardScaler()
        X_scaled = scaler.fit(data[['log_Area', 'log_Yield']])
        input_encoded[['log_Area', 'log_Yield']] = scaler.transform(input_encoded[['log_Area', 'log_Yield']])

    # -------------------------------
    # Predict and convert to tons
    # -------------------------------
    pred_log = model.predict(input_encoded)
    pred_log = np.clip(pred_log, 0, None)  # Prevent negative predictions
    pred_actual = np.expm1(pred_log)

    st.success(f\"\"\"
    🌾 **Predicted Production:** {pred_actual[0]:,.0f} tons\n
    🌱 **Crop:** {item_input}\n
    📍 **Region:** {area_input}\n
    📅 **Year:** {year_input}\n
    ⚙️ **Model Used:** {model_input}
    \"\"\")

    # -------------------------------
    # Production Trend for all models
    # -------------------------------
    hist_df = data[(data['Area_name'] == area_input) & (data['Item_name'] == item_input)].copy()
    hist_df['Production(tons)'] = np.expm1(hist_df['log_Production'])

    fig = go.Figure()

    # Actual production
    fig.add_trace(go.Scatter(
        x=hist_df['Year'],
        y=hist_df['Production(tons)'],
        mode='lines+markers',
        name='Actual',
        line=dict(color='black', width=3),
        marker=dict(size=6),
        hovertemplate="Year: %{x}<br>Actual: %{y:,.0f} tons"
    ))

    # Predictions from all models (hidden by default)
    for name, m in models_dict.items():
        model_data = data[(data['Area_name'] == area_input) & (data['Item_name'] == item_input)].copy()
        X_input = model_data[['Area', 'Item', 'Year', 'log_Area', 'log_Yield']]

        y_pred_log = m.predict(X_input)
        y_pred_actual = np.expm1(y_pred_log)

        fig.add_trace(go.Scatter(
            x=model_data['Year'],
            y=y_pred_actual,
            mode='lines+markers',
            name=f"{name} Prediction",
            visible='legendonly',  # 🔹 hidden until clicked in legend
            marker=dict(size=5),
            hovertemplate=f"Year: %{{x}}<br>{name}: %{{y:,.0f}} tons"
        ))

    # Future prediction for selected year
    for name, m in models_dict.items():
        input_encoded = pd.DataFrame({
            'Area': [mapping_area[area_input]],
            'Item': [mapping_item[item_input]],
            'Year': [year_input],
            'log_Area': [np.log1p(area_value)],
            'log_Yield': [np.log1p(yield_value)]
        })
        pred_future_log = m.predict(input_encoded)
        pred_future_actual = np.expm1(pred_future_log)

        fig.add_trace(go.Scatter(
            x=[year_input],
            y=[pred_future_actual[0]],
            mode='markers+text',
            name=f"{name} {year_input} Prediction",
            visible='legendonly',  # 🔹 also hidden until clicked
            # showlegend=False,
            marker=dict(size=10, symbol="star"),
            text=[f"{name}"],
            textposition="top center",
            hovertemplate=f"{name} Predicted: %{{y:,.0f}} tons"
        ))

    # Layout
    fig.update_layout(
        title=f"📈 Production Trend for {item_input} in {area_input} (All Models)",
        xaxis_title="Year",
        yaxis_title="Production (tons)",
        template="plotly_white",
        width=900,
        height=500,
        legend=dict(title="Legend", x=1, y=1),
    )

    st.plotly_chart(fig, use_container_width=True)


else:
    st.info("👈 Select inputs and click **Predict Production** from the sidebar to see results.")



"""

with open("app.py", "w") as f:
    f.write(code)

In [ ]:
!pip install streamlit pyngrok plotly


# Run

In [ ]:
# Need to get ngrok authtoken to run the streamlit app in local so get the token form the ngrok site

from pyngrok import ngrok

# Open a tunnel to port 8501
public_url = ngrok.connect(8501)
print(public_url)

# Run your Streamlit app
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0